In [1]:
import sqlite3
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import numpy as np

## 1. Load data

In [2]:
db = sqlite3.connect('../../data/MSN_technology.db')
cursor = db.cursor()
sql_text = 'SELECT * FROM IMAGES'
info_all = cursor.execute('SELECT * FROM IMAGES')

## 2. Generate models

In [3]:
model = InceptionResnetV1(pretrained='vggface2').eval()
mtcnn = MTCNN()

In [4]:
embeddings = []
embeddings_image = []

for (_id, _, _, _, _, _, _) in info_all:
	try:
		image = Image.open('../images/{}.jpg'.format(_id))
		image_cropped = mtcnn(image)
		image_embedding = model(image_cropped.unsqueeze(0)).detach().numpy().reshape((512))
		embeddings.append(image_embedding)
		embeddings_image.append(_id)
	except:
		pass

embeddings = np.array(embeddings)
embeddings_image = np.array(embeddings_image)

## 3. Save models

In [5]:
np.save('../../models/embeddings.npy', embeddings)
np.save('../../models/embeddings_image.npy', embeddings_image)